In [23]:
import os
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import trompy as tp
import pandas as pd
from scipy import stats

In [24]:
def get_number_cells (s2p_folder):
    iscell = np.load(os.path.join(s2p_folder, 'iscell.npy'))
    return sum(iscell[:,0])
    

In [25]:
def get_frames(events_file,s2p_length):
    df = pd.read_csv(events_file)

    pump_frames = list(df[df['Item1'] == 'pump_on']['Item2.Item2'])

    pump_frames=[frame for frame in pump_frames if frame < s2p_length-100] #removes event if too close to the end

    licks_frames = list (df[df['Item1']== 'Lick']['Item2.Item2'])
    
    return pump_frames, licks_frames

In [26]:
def filter_cell(x):

    t = np.arange(0,len(x)/10,0.1)
    filt = signal.butter(4, 1, 'low', fs=10, output='sos')
    filtered = signal.sosfilt(filt, x)
    
    return filtered
    

In [27]:
def get_activated_cells (s2p_folder,events_file):
    raw_F = np.load(os.path.join(s2p_folder, 'F.npy'))
    neu_F = np.load(os.path.join(s2p_folder, 'Fneu.npy'))
    iscell = np.load(os.path.join(s2p_folder, 'iscell.npy'))
#     stats=np.load(os.path.join(s2p_folder, 'stat.npy'), allow_pickle=True)
    
    s2p_length=len(raw_F[0,:])
    print(s2p_length)
    
    pump_frames, licks_frames= get_frames(events_file,s2p_length)
    
    cell_idx = [idx for idx,vals in enumerate(iscell) if vals[0]==1]
    
    activated_cells=0

    for i, cell in enumerate(cell_idx):
        x = raw_F[cell, :] - 0.7*(neu_F[cell, :])
        x = raw_F[cell, :]
        x = (x - np.mean(x))/np.std(x)
        x = filter_cell(x)
        
        
        snips = []
        pre=[]
        post=[]
        
        for p in pump_frames:
            snips.append(x[p-50:p+100])
            pre.append(np.mean(x[p-50:p]))
            post.append(np.mean(x[p:p+50]))
        result= stats.ttest_rel(pre, post)
        
#         activated=[]
#         for p in result[1]: 
#             if p < 0.05: 
#                 activated.append(p)
        
        
        #hit trials 
        hit=[]
        miss=[]
        licks=[]

        for p in pump_frames: 
            temp=[]
            for l in licks_frames: 
                if l > p and l < p + 100: 
                    temp.append(l)
                    
            if len(temp) > 0: 
                hit.append(p)
            if len(temp) == 0: 
                miss.append(p)
                
            licks.append(temp)
            
            
            
            
        #Snips and p-value hit trials        
                
        hit_snips=[]
        pre_hit=[]
        post_hit=[]
        
        for p in hit: 
            hit_snips.append(x[p-50:p+100])
            pre_hit.append(np.mean(x[p-50:p]))
            post_hit.append(np.mean(x[p:p+50]))
        hit_result= stats.ttest_rel(pre_hit, post_hit)
        
        #Snips and p-value missed trials
        
        miss_snips=[]
        pre_miss=[]
        post_miss=[]
        
        for p in miss: 
            miss_snips.append(x[p-50:p+100])
            pre_miss.append(np.mean(x[p-50:p]))
            post_miss.append(np.mean(x[p:p+50]))
        miss_result= stats.ttest_rel(pre_miss, post_miss)
      
    
        # snips and p-value of first lick in trial 
        temp_lick=[]
        for trial in licks:
            if len(trial) > 0: 
                temp_lick.append(trial[:1])
        first_lick=[trial for sublist in temp_lick for trial in sublist] 
        
        lick_snips=[]
        pre_lick=[]
        post_lick=[]
        
        for p in first_lick:
            lick_snips.append(x[p-50:p+100])
            pre_lick.append(np.mean(x[p-50:p]))
            post_lick.append(np.mean(x[p:p+50]))
        lick_result= stats.ttest_rel(pre_lick, post_lick)
        
        
            

#         print('all',result[1])
#         print('hit', hit_result[1])
        print('lick', lick_result[1])
        print('miss', miss_result[1])
    
#         pvalue=lick_result[1]
        
#     return pvalue
   
        
# paired t-test
#hit trials


In [40]:
animal="VGP310"
day="029"
date="20221108"
s2p_folder = "C:\\Users\\mko102\\UiT Office 365\\O365-Pipette 2 - Dokumenter\\People\\Mette\\GitHub\\2P\\Processed\\VGP_procd_data_22-12-05\\sub-{}\\ses-{}".format(animal, day)
events_file = "C:\\Users\\mko102\\UiT Office 365\\O365-Pipette 2 - Dokumenter\\People\\Mette\\GitHub\\2P\\Processed\\behav\\sub-{0}\\ses-{1}-{2}\\sub-{0}_ses-{1}_events.csv".format(animal, day, date)


In [41]:
get_number_cells(s2p_folder)

41.0

In [43]:
get_activated_cells(s2p_folder,events_file)


11559
lick 0.8432133743979955
miss 0.8293160723019313
lick 0.2741525077282527
miss 0.07318051895409128
lick 0.37123690789401775
miss 0.4159203497710776
lick 0.8516127129994251
miss 0.6961439112148022
lick 0.5789214678915819
miss 0.6931852121648809
lick 0.1419667429571832
miss 0.1042263292092449
lick 0.19883691136112727
miss 0.3268190592961411
lick 0.3314032077727276
miss 0.30720013793748524
lick 0.45992262127276506
miss 0.09636951908778953
lick 0.7239735928332359
miss 0.9003864791610526
lick 0.23485925925026765
miss 0.5217324046458628
lick 0.5011007991073813
miss 0.7222004771789634
lick 0.9258812861528857
miss 0.8035156543303323
lick 0.4088422769675676
miss 0.8687585635089661
lick 0.4389973467726428
miss 0.5479819600649154
lick 0.032537370515731225
miss 0.6516763253583994
lick 0.655912151240905
miss 0.7702488886413928
lick 0.5974940758392939
miss 0.3958788618142476
lick 0.04324132935149703
miss 0.5238221288558013
lick 0.12801795520277148
miss 0.94416528855035
lick 0.6607267361509928
mi